### dependence ###

In [1]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import hierarchical
from sklearn.cluster import estimate_bandwidth
from sklearn.metrics import calinski_harabaz_score
from sklearn.metrics import silhouette_score
from jieba.analyse import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# from Bio.Cluster import kcluster
# import synonyms
import pandas as pd
import jieba
jieba.add_word("报警人", freq=56, tag='nr')
jieba.add_word("系", freq=56, tag='r')
from math import isnan
import math
import numpy as np
import re
import csv
import jieba.posseg as pseg
import gensim

Building prefix dict from the default dictionary ...
Loading model from cache E:\local_temp\jieba.cache
Loading model cost 0.767 seconds.
Prefix dict has been built succesfully.


In [2]:
wx_from_text = gensim.models.KeyedVectors.load_word2vec_format('E:/chrome_download/2000000tencent.txt')
model = wx_from_text.wv

C:\Users\ym500\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


### main function ###

In [3]:
def deal_sklearn_model(model,
                       weight,
                       data,
                       df,
                       X,
                       print_details=0,
                       distance='euclidean'):
    kind = df[1].tolist()
    if print_details:
        print(model.labels_)
        print(len(model.labels_))
        details = {}
        for i in range(len(data)):
            if model.labels_[i] not in details:
                details[model.labels_[i]] = [[data[i]], [kind[i]]]
            else:
                details[model.labels_[i]][0].append(data[i])
                details[model.labels_[i]][1].append(kind[i])
        number = 0
        for key, list in details.items():
            this_kind = 0
            print("第%d类" % key)
            for i in range(len(list[0])):
                this_kind = this_kind + 1
                print("属于", list[1][i], ' ', list[0][i])
            print(this_kind)
            print()
            print()
            print()
            number += this_kind
        print("all_number == ", number)
    check_result(weight, model, distance)


# Hierarchical cluster
def AgglomerativeCluster(weight, data, df, X, n_cluster, print_details=0):
    print("Agglomerative算法 n =",n_cluster)
    agg_model = AgglomerativeClustering(n_clusters=n_cluster).fit(weight)
    deal_sklearn_model(agg_model, weight, data, df, X, print_details)
    return agg_model.labels_


# K-Means 算法
def K_Means(weight, data, df, X, n_cluster, print_details=0):
    print("K_Means算法 n =",n_cluster)
    iter_number = 700
    kmodel = KMeans(n_clusters=n_cluster,
                    n_init=10,
                    max_iter=iter_number,
                    init='k-means++',
                    n_jobs=-1).fit(weight)
    deal_sklearn_model(kmodel, weight, data, df, X, print_details)
    return kmodel.labels_


# DBSCAN 算法
def AAedion_DBSCAN(weight, data, df, X, print_details=0, distance='euclidean'):
    print("DBSCAN算法")
    DBmodel = DBSCAN(eps=0.5, min_samples=10, metric=distance,
                     n_jobs=-1).fit(weight)
    deal_sklearn_model(DBmodel, weight, data, df, X, print_details, distance)
    print("DBSCAN算法完成")
    return DBmodel.labels_


def AAedion_Birch(weight, data, df, X, n_cluster,print_details=0, distance='euclidean'):
    print('Birch算法 n =',n_cluster)
    Birch_model = Birch(n_clusters=n_cluster).fit(weight)
    deal_sklearn_model(
            Birch_model,
            weight,
            data,
            df,
            X,
            print_details,
        )
    return Birch_model.labels_


def AAedion_SpectralClustering(weight,
                               data,
                               df,
                               X,
                               print_details=0,
                               distance='euclidean',
                               n_jobs=-1):
    print('SpectralClustering算法')
    Spe_model = SpectralClustering(n_clusters=n_clusters).fit(weight)
    deal_sklearn_model(
        Spe_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return Spe_model.labels_


def AAedion_AffinityPropagation(weight,
                                data,
                                df,
                                X,
                                print_details=0,
                                distance='euclidean'):
    print('AffinityPropagatio算法')
    Aff_model = AffinityPropagation(damping=0.5,
                                    max_iter=200,
                                    convergence_iter=15,
                                    copy=True,
                                    preference=None,
                                    affinity='euclidean',
                                    verbose=False).fit(weight)
    deal_sklearn_model(
        Aff_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return Aff_model.labels_


def AAedion_Meanshift(weight,
                      data,
                      df,
                      X,
                      print_details=0,
                      distance='euclidean'):
    print('MeanShift算法')
    #     bandwidth = estimate_bandwidth(weight, quantile=0.3,n_jobs=-1)
    #    print(bandwidth)
    MeanShift_model = MeanShift(bandwidth=1,
                                seeds=None,
                                bin_seeding=True,
                                min_bin_freq=1,
                                cluster_all=False,
                                n_jobs=-1).fit(weight)
    print('Meanshfit finish')
    deal_sklearn_model(
        MeanShift_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return MeanShift_model.labels_


def check_result(weight, model, distance):
    #print("共%d类" %model.labels_)
    print("result of harabaz_score: ",
          calinski_harabaz_score(weight, model.labels_))
    print("result of silhouette_score: ",
          silhouette_score(weight, model.labels_, metric='euclidean'))


def print_TF_IDF_values():
    for i in range(len(weight)):  # 打印每类文本的tf-idf词语权重
        print("-------这里输出第", i, u"类文本的词语tf-idf权重------")
        # for j in range(len(word)):
        print(list(zip(all_word, weight[i])))

### stop word ###

In [4]:
    # 人工停用词表
    special_word = ['称', '，', '。', '在', '年', '月', '(', '（','民警','报警','观音桥','称有','称一','称车','称其','接警','人称']

    # 加载停用词表
    Stopword_FilePath = "../stopWord.txt"
    stopWordList = pd.read_csv(Stopword_FilePath, sep="\r")
    for word in special_word:
        stopWordList.loc[stopWordList.size] = word
    print("停用词表的大小是", stopWordList.size)

    # 加载数据
    FilePath = "../data.csv"
    df = pd.read_csv(FilePath,header=None)

    # 停用词过滤
    data = df[5]
    kind = df[2]
    X = []
    delete = []
    record = {}
    person_number = 0
    place_number = 0
    pronoun_number = 0
    beside_number = 0
    all_number = 0
    flag_record = {}
    flag_form = {}
    flag_result = {}
    data_keywords = []
#     data_new = []
#     kind_new = []
#     k = 0
#     for i in range(len(data)):
#         try:
#             if isnan(float(data[i])):
#                 k += 1
#         except ValueError:    
#              data_new.append(data[i])
#              kind_new.append(kind[i])
#     data = data_new
#     kind = kind_new
    for i in range(len(data)):
        cut_result = []
        person_name = []
        place_name = []
        pronoun = []
        beside = []
        # 按词性过滤
        #print(data[i])
        k = pseg.cut(data[i])
        for word, flag in k:
            flag_form[word] = flag
            if flag not in flag_record:
                flag_record[flag] = 1
            else:
                flag_record[flag] += 1
                
            if flag == 'nr' or flag == 'nr1' or flag == 'nr2':
                person_name.append(word)
                person_number += 1
                continue
            if flag == 'ns' or flag == 'nz':
                place_name.append(word)
                place_number += 1
                continue
            if flag == 'r' or flag == 'rr' or flag == 'rz':
                pronoun.append(word)
                pronoun_number += 1
                continue
            if flag == 'w' or flag == 'h' or flag == 'k' or flag == 'xx' \
                    or flag == 'o' or flag == 'u' or flag == 'm' or flag == 'd' \
                    or flag == 'f':
                beside.append(word)
                beside_number += 1
                continue
        # 提取关键词列表
        for keyword, weight in extract_tags(data[i], withWeight=True):
            if keyword in person_name or keyword in place_name or keyword in pronoun or keyword in beside:
                all_number += 1
                delete.append(keyword)
                continue
            # if flag_form[keyword] not in flag_result:
            #     flag_result[flag_form[keyword]] = [keyword]
            # else:
            #     flag_result[flag_form[keyword]].append(keyword)
            cut_result.append(keyword)
        # print(data[i],": 关键词",cut_result)

        # 用停用词库筛选关键词列表
        temp = ""
        for j in cut_result:
            if j in stopWordList.values or re.search("[0-9]+", j) is not None \
                    or re.search("[a-z]+", j) is not None or re.search("[A-Z]+", j) is not None:
                if j not in delete:
                    delete.append(j)
            else:
                if temp == '':
                    temp = j
                else:
                    temp = temp + " " + j
                if j in record:
                    record[j] = record[j] + 1
                else:
                    record[j] = 1
        X.append(temp)
        data_keywords.append(temp)
    flag_record = sorted(flag_record.items(), key=lambda item: item[1], reverse=True)
    result_X = []
    for string in X:
        temp = ''
        for j in string.split(' '):
            if j == '':
                continue
            if record[j] < 5:
                continue
            temp = temp + " " + j
            # print(j)
        result_X.append(temp)
    X_str = np.array(result_X)
    record = sorted(record.items(), key=lambda item: item[1], reverse=True)
    print('finish')
#     print("被清除的词有", delete)
#     print("最终词典", record)
#     print("过滤结果", X_str)
#     print('person_number: ', person_number)
#     print('place_number: ', place_number)
#     print('pronoun_number: ', pronoun_number)
#     print('beside_number: ', beside_number)
#     print('all_number: ', all_number)
#     print('flag_record: ', flag_record)

    # for w, f in flag_result.items():
    #     print('%s类' % w)
    #     for str in f:
    #         print(str)
    #     print()
    #     print()
    #     print()

停用词表的大小是 2085
finish


### construction TF-IDF ###

#### synonym word replace ####

In [5]:
# preprocessing
data = result_X
synonym_map = {}
no_synonym_words_count = 0
for i in range(len(data)):
    for string in data[i].split(' '):
        if string not in synonym_map:
            try:
                synonym_list = model.most_similar(string)
                synonym_map[string] = string
                for item in synonym_list:
                    synonym_word = item[0]
                    if synonym_word not in synonym_map:
                        synonym_map[synonym_word] = string
                        print(synonym_word,' to ',string)
#                         print('add ',synonym_word,'to ',string)
            except Exception as e:
                    no_synonym_words_count += 1
#                 print('no word ',string)
# replace
result = []
for i in range(len(data)):
    temp = ""
    for string in data[i].split(' '):
        if string in  synonym_map:
            replace_string = synonym_map[string]
            # print(string," change to ",replace_string)
        else:
            replace_string = string
        if temp != "":
            temp = temp + " " + replace_string
        else:
            temp = replace_string
    result.append(temp)
X_str = np.array(result)

困住  to  困在
困于  to  困在
被困  to  困在
困入  to  困在
关在  to  困在
围困  to  困在
囚困  to  困在
囚在  to  困在
困着  to  困在
囚禁  to  困在
在水中  to  水中
水里  to  水中
水底  to  水中
入水  to  水中
河水中  to  水中
海水中  to  水中
水面  to  水中
湖中  to  水中
在水  to  水中
水中漂浮  to  水中
时间——  to  时间
时间时间  to  时间
时间里  to  时间
时间……  to  时间
时间吧  to  时间
时间不  to  时间
时间很长  to  时间
把时间  to  时间
时间时  to  时间
时间啊  to  时间
长期  to  长时间
较长时间  to  长时间
长时期  to  长时间
过久  to  长时间
时间过长  to  长时间
太久  to  长时间
时间太长  to  长时间
很长时间  to  长时间
时间长  to  长时间
短时间  to  长时间
据传  to  据说
听说  to  据说
号称  to  据说
据称  to  据说
大家都知道  to  据说
据闻  to  据说
估计  to  据说
坊间传闻  to  据说
传说  to  据说
而且  to  据说
送电  to  停电
停电通知  to  停电
停水  to  停电
停电区域  to  停电
因停电  to  停电
停电施工  to  停电
线路检修  to  停电
停电计划  to  停电
停电检修  to  停电
停电作业  to  停电
造成  to  导致
致使  to  导致
导至  to  导致
由于  to  导致
容易导致  to  导致
直接原因  to  导致
严重  to  导致
可能导致  to  导致
主要原因  to  导致
加剧  to  导致
引起  to  引发
可能引发  to  引发
引发出  to  引发
会引起  to  引发
引来  to  引发
产生  to  引发
惹起  to  引发
因  to  称因
称与  to  称因
发布公告  to  称因
称未  to  称因
公告称  to  称因
疑因  to  称因
称  to  称因
称要 

倒车时  to  倒车
变道  to  倒车
后车  to  倒车
超车  to  倒车
变换车道  to  倒车
前车  to  倒车
打转向灯  to  倒车
并线  to  倒车
打方向盘  to  倒车
后方车辆  to  倒车
查询  to  查看
察看  to  查看
马上查看  to  查看
可以查看  to  查看
快速查看  to  查看
查找  to  查看
产看  to  查看
仔细查看  to  查看
查阅  to  查看
扫描  to  查看
其他车辆  to  车辆
其它车辆  to  车辆
行驶车辆  to  车辆
车辆行驶  to  车辆
行驶的车辆  to  车辆
及车辆  to  车辆
车辆情况  to  车辆
如车辆  to  车辆
机动车  to  车辆
车俩  to  车辆
否认  to  承认
不承认  to  承认
承认事实  to  承认
直接承认  to  承认
事实  to  承认
声称  to  承认
坦承  to  承认
公开承认  to  承认
矢口否认  to  承认
坚称  to  承认
逃逸者  to  逃逸
驾车逃逸  to  逃逸
弃车逃逸  to  逃逸
肇事  to  逃逸
肇事逃逸  to  逃逸
肇事人  to  逃逸
肇事者  to  逃逸
逃离事故现场  to  逃逸
逃窜  to  逃逸
逃离现场  to  逃逸
赔偿金  to  赔偿
索赔  to  赔偿
赔偿损失  to  赔偿
赔偿费  to  赔偿
赔偿金额  to  赔偿
精神损失费  to  赔偿
经济赔偿  to  赔偿
全额赔偿  to  赔偿
精神损失  to  赔偿
精神赔偿  to  赔偿
突然发生  to  发生
已发生  to  发生
发生过  to  发生
再次发生  to  发生
会发生  to  发生
经常发生  to  发生
可能发生  to  发生
很可能发生  to  发生
前发生  to  发生
其发生  to  发生
车主们  to  车主
女车主  to  车主
宝马车主  to  车主
奔驰车主  to  车主
车主会  to  车主
车友  to  车主
汽车车主  to  车主
私家车主  to  车主
爱车  to  车主
4s店  to  车主
直接回复  to  回复
微信回复 

几十人  to  十几人
十多人  to  十几人
上百人  to  十几人
近百人  to  十几人
数百人  to  十几人
二百多人  to  十几人
几百人  to  十几人
二十人  to  十几人
上千人  to  十几人
百余人  to  十几人
餐厅地址  to  地址
店铺地址  to  地址
店地址  to  地址
商家地址  to  地址
门店地址  to  地址
分店地址  to  地址
总店地址  to  地址
公司地址  to  地址
联系地址  to  地址
address  to  地址
详尽  to  详细
详细地  to  详细
详细说明  to  详细
更详细  to  详细
详细的说明  to  详细
简要  to  详细
详实  to  详细
具体  to  详细
详细介绍  to  详细
详细的介绍  to  详细
医院就医  to  医院
大医院  to  医院
诊所  to  医院
人民医院  to  医院
在医院  to  医院
医院看病  to  医院
医生  to  医院
医院的医生  to  医院
急诊  to  医院
医院医生  to  医院
国内外  to  国际
国际性  to  国际
国际国内  to  国际
中国国际  to  国际
亚太  to  国际
国际高  to  国际
全球  to  国际
国内国际  to  国际
国际和国内  to  国际
国内  to  国际
前妻  to  前夫
离婚后  to  前夫
离了婚  to  前夫
离婚之后  to  前夫
离婚以后  to  前夫
丈夫  to  前夫
离婚  to  前夫
前婆婆  to  前夫
继母  to  前夫
继父  to  前夫
家里  to  家中
家里的  to  家中
家中客厅  to  家中
家内  to  家中
家中物品  to  家中
家中成员  to  家中
家中生活  to  家中
在家里  to  家中
不给  to  不让
没让  to  不让
不想  to  不让
逼着  to  不让
要让  to  不让
没办法  to  不让
不许  to  不让
别让  to  不让
不肯  to  不让
怕  to  不让
兴奋  to  激动
心情激动  to  激动
很激动  to  激动
激动地  to  

管辖范围  to  管辖
其管辖  to  管辖
所辖  to  管辖
划归  to  管辖
管辖权  to  管辖
统辖  to  管辖
行政管辖  to  管辖
辖地  to  管辖
管辖区  to  管辖
辖下  to  管辖
今日  to  昨日
早间  to  昨日
前日  to  昨日
12月8日  to  昨日
12月14日  to  昨日
12月18日  to  昨日
上周五  to  昨日
近几日  to  昨日
不明白  to  不通
难通  to  不通
想不通  to  不通
窍  to  不通
不太通  to  不通
通  to  不通
想不明白  to  不通
通了  to  不通
不通畅  to  不通
其价值  to  价值
真正的价值  to  价值
市场价值  to  价值
实际价值  to  价值
物质价值  to  价值
其他价值  to  价值
价值意义  to  价值
使用价值  to  价值
经济价值  to  价值
潜在价值  to  价值
应看  to  需看
要看  to  需看
需视  to  需看
看该  to  需看
配合情况  to  需看
需要关注  to  需看
要视  to  需看
不好确定  to  需看
如有效  to  需看
需待  to  需看
短息  to  短信
短信息  to  短信
一条短信  to  短信
短信内容  to  短信
电话和短信  to  短信
手机短信  to  短信
微信信息  to  短信
微信消息  to  短信
陌生号码  to  短信
短信里  to  短信
是因为  to  原因
其原因  to  原因
根本原因  to  原因
重要原因  to  原因
原因之一  to  原因
原因是什么  to  原因
最主要原因  to  原因
究其原因  to  原因
因为  to  原因
找住的  to  现找
找便宜  to  现找
定住宿  to  现找
找宾馆  to  现找
现去  to  现找
先预定  to  现找
找旅馆  to  现找
找住宿  to  现找
找酒店  to  现找
找客栈  to  现找
摩托车  to  摩托
三轮摩托  to  摩托
骑摩托  to  摩托
三蹦子  to  摩托
小摩托车  to  摩托
机车  to  摩

余人  to  多人
100多人  to  多人
300人  to  多人
40人  to  多人
20人  to  多人
120人  to  多人
8人  to  多人
60人  to  多人
9人  to  多人
6人  to  多人
网贷平台  to  网贷
p2p平台  to  网贷
p2p  to  网贷
网络借贷平台  to  网贷
贷平台  to  网贷
现金贷  to  网贷
p2p网贷  to  网贷
网贷公司  to  网贷
p2p网贷平台  to  网贷
校园贷  to  网贷
言语攻击  to  辱骂
侮辱谩骂  to  辱骂
谩骂  to  辱骂
语言攻击  to  辱骂
殴打  to  辱骂
言语辱骂  to  辱骂
辱骂他人  to  辱骂
侮辱性  to  辱骂
漫骂  to  辱骂
侮辱  to  辱骂
问题  to  解决
问题解决  to  解决
彻底解决  to  解决
解决目前  to  解决
难题  to  解决
最终解决  to  解决
解决问题  to  解决
解决好  to  解决
完美解决  to  解决
无法解决  to  解决
亲人  to  家人
家人和朋友  to  家人
父母家人  to  家人
家人朋友  to  家人
一家人  to  家人
亲友  to  家人
全家人  to  家人
父母  to  家人
跟家人  to  家人
其他家人  to  家人
配图  to  网络
于网  to  网络
及网络  to  网络
图片  to  网络
以及网络  to  网络
侵删  to  网络
图片来源  to  网络
图源  to  网络
网路  to  网络
必删  to  网络
情色  to  色情
色情内容  to  色情
淫秽  to  色情
性暗示  to  色情
暴力色情  to  色情
低俗  to  色情
色情暴力  to  色情
se情  to  色情
色情电影  to  色情
软色情  to  色情
提供服务  to  服务
服务提供  to  服务
提供的服务  to  服务
专业服务  to  服务
服务……  to  服务
增值服务  to  服务
产品服务  to  服务
附加服务  to  服务
支持服务  to  服务
服务产品  to  服务
朋友借钱  to  借钱

不在场  to  在场
再场  to  在场
而场  to  在场
上场  to  在场
场上  to  在场
场下  to  在场
两人场  to  在场
但场  to  在场
和场  to  在场
的场  to  在场
铁了心  to  执意
一意孤行  to  执意
不答应  to  执意
决意  to  执意
不同意  to  执意
劝说  to  执意
非要  to  执意
死活  to  执意
撬锁  to  开锁
钥匙开锁  to  开锁
开门锁  to  开锁
开车锁  to  开锁
车锁  to  开锁
电子锁  to  开锁
换锁  to  开锁
开锁师傅  to  开锁
开锁方式  to  开锁
锁匠  to  开锁
工人们  to  工人
其他工人  to  工人
建筑工人  to  工人
工人多  to  工人
工厂工人  to  工人
工人工作  to  工人
施工工人  to  工人
工厂的工人  to  工人
工厂里  to  工人
临时工人  to  工人
选择购买  to  购买
再购买  to  购买
去购买  to  购买
想要购买  to  购买
购买后  to  购买
直接购买  to  购买
想购买  to  购买
购买价格  to  购买
准备购买  to  购买
再次购买  to  购买
活动地点  to  地点
时间和地点  to  地点
时间地点  to  地点
时间及地点  to  地点
选在  to  地点
时间与地点  to  地点
演出地点  to  地点
拍摄地点  to  地点
会议地点  to  地点
整体做工  to  做工
做工细节  to  做工
细节做工  to  做工
做工工艺  to  做工
材质做工  to  做工
做工用料  to  做工
做工质量  to  做工
做工质感  to  做工
做工细致  to  做工
外观做工  to  做工
认定工伤  to  工伤
工伤认定  to  工伤
工伤事故  to  工伤
工伤待遇  to  工伤
工伤保险  to  工伤
因工受伤  to  工伤
发生工伤  to  工伤
申请工伤认定  to  工伤
公伤  to  工伤
工伤保险条例  to  工伤
药物治疗  to  治疗
治疗中  to  治疗
相关治疗  to  治疗
治疗方式

其他房屋  to  房屋
居住的房屋  to  房屋
以及房屋  to  房屋
新建房屋  to  房屋
楼房  to  房屋
房屋结构  to  房屋
该房  to  房屋
居住房屋  to  房屋
买卖交易  to  买卖
卖买  to  买卖
交易买卖  to  买卖
买进卖出  to  买卖
买卖自由  to  买卖
做买卖  to  买卖
商品买卖  to  买卖
买和卖  to  买卖
大宗买卖  to  买卖
市场买卖  to  买卖
接电话  to  来电
陌生来电  to  来电
回电  to  来电
接听电话  to  来电
手机来电  to  来电
来电信息  to  来电
语音电话  to  来电
说刚  to  称刚
其刚  to  称刚
翁先生  to  称刚
丁女士  to  称刚
竟称  to  称刚
刘某刚  to  称刚
邹女士  to  称刚
董女士  to  称刚
莫女士  to  称刚
已受理  to  受理
立案  to  受理
直接受理  to  受理
正式受理  to  受理
受理申请  to  受理
立案受理  to  受理
登记受理  to  受理
办结  to  受理
依法受理  to  受理
受理案件  to  受理
家暴  to  家庭暴力
遭受家庭暴力  to  家庭暴力
遭遇家庭暴力  to  家庭暴力
家庭冷暴力  to  家庭暴力
家暴受害者  to  家庭暴力
家庭暴力问题  to  家庭暴力
受暴者  to  家庭暴力
精神暴力  to  家庭暴力
实施家庭暴力  to  家庭暴力
路过此地  to  路过
偶然路过  to  路过
路过了  to  路过
偶尔路过  to  路过
路经  to  路过
过路  to  路过
只是路过  to  路过
匆匆路过  to  路过
路过看到  to  路过
前去  to  前往
前往当地  to  前往
赶往  to  前往
去往  to  前往
独自前往  to  前往
抵达  to  前往
赶赴  to  前往
自行前往  to  前往
返回  to  前往
赴  to  前往
各种首饰  to  首饰
饰品  to  首饰
珠宝  to  首饰
其他首饰  to  首饰
手饰  to  首饰
珠宝首饰  to  首饰
耳环  to  首饰
金饰 

朋友见面  to  见面
再次见面  to  见面
相见  to  见面
见了面  to  见面
再见面  to  见面
正式见面  to  见面
见面后  to  见面
碰面  to  见面
见过面  to  见面
见个面  to  见面
区域内  to  区内
区外  to  区内
区域  to  区内
区  to  区内
区域外  to  区内
区的  to  区内
集中区域  to  区内
域内  to  区内
园区内  to  区内
集中区  to  区内
今年  to  去年
10月份  to  去年
今年年初  to  去年
2016年  to  去年
9月份  to  去年
去年年底  to  去年
8月份  to  去年
年初  to  去年
7月份  to  去年
5月份  to  去年
活动中  to  活动
活动会  to  活动
活动时  to  活动
一系列活动  to  活动
各种活动  to  活动
各类活动  to  活动
活动不断  to  活动
活动后  to  活动
活动形式  to  活动
活动的形式  to  活动
未成人  to  未成年人
未成年学生  to  未成年人
未成年  to  未成年人
未成年人的保护  to  未成年人
未满18周岁  to  未成年人
未成年人犯罪  to  未成年人
保护未成年人  to  未成年人
未成年犯罪  to  未成年人
未成年人权益  to  未成年人
18岁以下未成年人  to  未成年人
日下午  to  日晚
日上午  to  日晚
日中午  to  日晚
月17  to  日晚
月6  to  日晚
月11  to  日晚
月13  to  日晚
月14  to  日晚
月27  to  日晚
月9  to  日晚
中心一  to  中心
三个中心  to  中心
中心建设  to  中心
服务中心  to  中心
综合中心  to  中心
四大中心  to  中心
两个中心  to  中心
三大中心  to  中心
中心规划  to  中心
中心功能  to  中心
更多详情  to  详情
详细内容  to  详情
具体信息  to  详情
详情请  to  详情
请见  to  详情
活动详情  to  详情
可查看  to  详情
详情请点击  to  详情

出门时  to  出门
出门儿  to  出门
出门逛街  to  出门
出门前  to  出门
外出  to  出门
出门去  to  出门
孩子出门  to  出门
出门办事  to  出门
出门购物  to  出门
出们  to  出门
未缴  to  未交
欠交  to  未交
已缴  to  未交
没缴  to  未交
已交  to  未交
补交  to  未交
欠缴  to  未交
交清  to  未交
漏交  to  未交
缴清  to  未交
死亡……  to  死亡
最终死亡  to  死亡
死亡…  to  死亡
死去  to  死亡
失去生命  to  死亡
人的死亡  to  死亡
造成死亡  to  死亡
致死  to  死亡
死亡者  to  死亡
自然死亡  to  死亡
称卖  to  称买
转买  to  称买
因买  to  称买
因购买  to  称买
以买  to  称买
其买  to  称买
其卖  to  称买
称用  to  称买
竟买  to  称买
强暴  to  强奸
被强奸  to  强奸
轮奸  to  强奸
奸杀  to  强奸
猥亵  to  强奸
奸污  to  强奸
施暴  to  强奸
强奸未遂  to  强奸
奸淫  to  强奸
强jian  to  强奸
临时停车费  to  停车费
车位费  to  停车费
车位管理费  to  停车费
停车场收费  to  停车费
存车费  to  停车费
停车收费  to  停车费
车位租金  to  停车费
停车管理费  to  停车费
租车位  to  停车费
物业费  to  停车费
离世  to  去世
过世  to  去世
逝世  to  去世
因病去世  to  去世
病逝  to  去世
辞世  to  去世
去逝  to  去世
离开人世  to  去世
病故  to  去世
撒手人寰  to  去世
真实意图  to  意图
用意  to  意图
战略意图  to  意图
动机  to  意图
本意  to  意图
主要意图  to  意图
意在  to  意图
借此  to  意图
很显然  to  意图
小箱子  to  箱子
大箱子  to  箱子
箱子里  to  箱子
木箱子  to  箱子
木箱  to  箱子
皮箱  to 

In [6]:
print(len(synonym_map))

6262


#### construction TF-IDF based on synonym words ####

In [7]:
vectorizer = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值
X = vectorizer.fit_transform(X_str)  # 将文本转为词频矩阵
tfidf = transformer.fit_transform(X)  # 计算tf-idf，
all_word = vectorizer.get_feature_names()  # 获取词袋模型中的所有词语
weight = tfidf.toarray()  # 将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

# print_TF_IDF_values()
print("weight shape:", weight.shape)
print("word shape:", len(all_word))

weight shape: (6481, 611)
word shape: 611


### Cluster ###

#### cluster ####

In [8]:
def get_CHI(types):
    corpus = X_str  #数据类
    words = all_word  #特征词
    cluster_result = types
    cluster_map = {}
    for i in range(len(cluster_result)):
        string = cluster_result[i]
        if string in cluster_map:
            cluster_map[string][0].append(data[i])
            cluster_map[string][1].append(i)
            cluster_map[string][2] += 1
        else:
            cluster_map[string] = [[data[i]], [i], 1]
    delete_map = {}
    
    for key,value in cluster_map.items():
        if value[2] <= len(data)//100:
            delete_map[key] = 1
            
    for key,value in delete_map.items():
        cluster_map.pop(key)
        
    cluster_map_save = cluster_map
    cluster_map = sorted(cluster_map.items(),
                         key=lambda item: item[1][2],
                         reverse=True)
    print("聚类总情况数: ", len(cluster_map))
    print("最大类的样本数为：", cluster_map[0][1][2])

    types_words_ratio = {}
    types_words_N = len(corpus)
    types_words_A = {}  #属于某类别ci也含有特征词的文本数目
    types_words_B = {}  #不属于某类别ci也含有特征词的文本数目
    types_words_C = {}  #属于某类别ci但不含有特征词的文本数目
    types_words_D = {}  #不属于某类别ci也不含有特征词的文本数目

    for i in range(len(cluster_map)):
        t = cluster_map[i][0]
        for w in words:
            pair = t + w
            for i in range(len(corpus)):
                if types[i] == t and w in corpus[i]:
                    types_words_A[pair] = types_words_A.get(pair, 0) + 1
                    continue
                if types[i] != t and w in corpus[i]:
                    types_words_B[pair] = types_words_B.get(pair, 0) + 1
                    continue
                if types[i] == t and w not in corpus[i]:
                    types_words_C[pair] = types_words_C.get(pair, 0) + 1
                    continue
                if types[i] != t and w not in corpus[i]:
                    types_words_D[pair] = types_words_D.get(pair, 0) + 1
                    continue

    words_suit_types = {}  #CHI值所对应的类
    words_suit_marks = {}  #CHI值
    for word in words:
        for i in range(len(cluster_map)):
            t = cluster_map[i][0]
            pair = t + word
            CHI = types_words_N * math.pow(
                (types_words_A.get(pair, 0) * types_words_D.get(pair, 0) -
                 types_words_B.get(pair, 0) * types_words_C.get(pair, 0)), 2)
            CHI = CHI / (
                (types_words_A.get(pair, 0) + types_words_C.get(pair, 0)) *
                (types_words_B.get(pair, 0) + types_words_D.get(pair, 0)))
            CHI = CHI / (
                (types_words_A.get(pair, 0) + types_words_B.get(pair, 0)) *
                (types_words_C.get(pair, 0) + types_words_D.get(pair, 0)))
            if CHI > words_suit_marks.get(word, 0):
                words_suit_marks[word] = CHI
                words_suit_types[word] = t
        types = words_suit_types[word]
        marks = words_suit_marks[word]
    return words_suit_marks


def clustering(print_details, weight):
    word_to_chi = {}
    for i in range(17, 22):
        n_cluster = i
        KMeans_labels = K_Means(weight, data, df, X, n_cluster, print_details)
        Birch_labels = AAedion_Birch(weight, data, df, X, n_cluster,
                                     print_details)
        Hierarchical_labels = AgglomerativeCluster(weight, data, df, X,
                                                   n_cluster, print_details)
        all_labels = [
            KMeans_labels,  #DS_labels, 
            Hierarchical_labels,
            Birch_labels
        ]
        cluster_result = deal(all_labels)
        CHI = get_CHI(cluster_result)
        for key,value in CHI.items():
            if key in word_to_chi:
                word_to_chi[key] = word_to_chi[key] + value
            else:
                word_to_chi[key] = value
    CHI_value_map = sorted(word_to_chi.items(),
                         key=lambda item: item[1],
                         reverse=True)
    print(CHI_value_map[0:20])
    return all_labels,CHI_value_map


# KMeans_labels = K_Means(weight, data, df, X, print_details)
# DBSCAN_labels = AAedion_DBSCAN(weight, data, df, X, print_details)
# Brich_labels = AAedion_Birch(weight, data, df, X, print_details)
# SpectralClustering_labels = AAedion_SpectralClustering(weight, data, df, X, print_details)

### can't use

# AAedion_AffinityPropagation(weight, data, df, X, 1)
# AAedion_Meanshift(weight, data, df, X, 1)

#### change weight ####

In [9]:
class find_CHI_index_Error(Exception):
    pass


def get_CHI_index(CHI_value, word):
    index = 0
    for item in CHI_value:
        index += 1
        if index >= len(CHI_value) * 9 // 10:
            break
        if item[0] == word:
            return 1 - index / len(CHI_value)
    raise find_CHI_index_Error(word + 'can not find in CHI_value')
    print(word,'decrease')
    return -0.2


def deal(all_labels):
    cluster_result = []
    for i in range(0, len(all_labels[0])):
        cluster_result.append('')
    for labels in all_labels:
        for i in range(0, len(labels)):
            cluster_result[i] = cluster_result[i] + " " + str(labels[i])
    return cluster_result


def Change(all_labels, weight, CHI_value):
    cluster_result = deal(all_labels)
    cluster_map = {}
    for i in range(len(cluster_result)):
        string = cluster_result[i]
        if string in cluster_map:
            cluster_map[string][0].append(data[i])
            cluster_map[string][1].append(i)
            cluster_map[string][2] += 1
        else:
            cluster_map[string] = [[data[i]], [i], 1]

    cluster_map = sorted(cluster_map.items(),
                         key=lambda item: item[1][2],
                         reverse=True)
    print("聚类总情况数: ", len(cluster_map))
    # for item in cluster_map:
    #     print("所属:  ", item[0], "数量： ", item[1][2])
    goal_words = {}
    for i in range(0, len(cluster_map) // 2**len(all_labels)):
        #         if cluster_map[i][1][2] < len(data)/40:
        #             continue
        print(i, cluster_map[i][1][2])
        wordcount_record = {}
        for item in cluster_map[i][1][1]:
            #     print(data_keywords[item])
            for word in data_keywords[item].split(" "):
                if word == ' ' or word == '':
                    continue
                if word in wordcount_record:
                    wordcount_record[word] += 1
                else:
                    wordcount_record[word] = 1
        wordcount_record = sorted(wordcount_record.items(),
                                  key=lambda item: item[1],
                                  reverse=True)
        # increase the first and second word
        alpha = 1.5
        for item in wordcount_record[0:2]:
            increase_rate = 1 + alpha * get_CHI_index(
                CHI_value , item[0]) * (item[1] / cluster_map[i][1][2])
            print(item[0], " ", item[1], " increase rate: ", increase_rate)
            if item[0] not in goal_words:
                weight = changeWeight(weight, all_word, item[0], increase_rate,
                                      0)
                goal_words[item[0]] = 1
    return weight


# def Change_Chi(all_labels, weight):
#     types = deal(all_labels)  #类list
#     corpus = data  #数据类
#     words = all_word  #特征词

#     cluster_result = types
#     cluster_map = {}
#     for i in range(len(cluster_result)):
#         string = cluster_result[i]
#         if string in cluster_map:
#             cluster_map[string][0].append(data[i])
#             cluster_map[string][1].append(i)
#             cluster_map[string][2] += 1
#         else:
#             cluster_map[string] = [[data[i]], [i], 1]
#     cluster_map_save = cluster_map
#     cluster_map = sorted(cluster_map.items(),
#                          key=lambda item: item[1][2],
#                          reverse=True)
#     print("聚类总情况数: ", len(cluster_map))
#     print("最大类的样本数为：", cluster_map[0][1][2])

#     types_words_ratio = {}
#     types_words_N = len(corpus)
#     types_words_A = {}  #属于某类别ci也含有特征词的文本数目
#     types_words_B = {}  #不属于某类别ci也含有特征词的文本数目
#     types_words_C = {}  #属于某类别ci但不含有特征词的文本数目
#     types_words_D = {}  #不属于某类别ci也不含有特征词的文本数目

#     for i in range(len(cluster_map)):
#         t = cluster_map[i][0]
#         for w in words:
#             pair = t + w
#             for i in range(len(corpus)):
#                 if types[i] == t and w in corpus[i]:
#                     types_words_A[pair] = types_words_A.get(pair, 0) + 1
#                     continue
#                 if types[i] != t and w in corpus[i]:
#                     types_words_B[pair] = types_words_B.get(pair, 0) + 1
#                     continue
#                 if types[i] == t and w not in corpus[i]:
#                     types_words_C[pair] = types_words_C.get(pair, 0) + 1
#                     continue
#                 if types[i] != t and w not in corpus[i]:
#                     types_words_D[pair] = types_words_D.get(pair, 0) + 1
#                     continue

#     words_suit_types = {}  #CHI值所对应的类
#     words_suit_marks = {}  #CHI值
#     words_types_marks = {}  #不同类的CHI排序
#     for i in range(len(cluster_map)):
#         words_types_marks[cluster_map[i][0]] = {}

#     for word in words:
#         for i in range(len(cluster_map)):
#             t = cluster_map[i][0]
#             pair = t + word
#             CHI = types_words_N * math.pow(
#                 (types_words_A.get(pair, 0) * types_words_D.get(pair, 0) -
#                  types_words_B.get(pair, 0) * types_words_C.get(pair, 0)), 2)
#             CHI = CHI / (
#                 (types_words_A.get(pair, 0) + types_words_C.get(pair, 0)) *
#                 (types_words_B.get(pair, 0) + types_words_D.get(pair, 0)))
#             CHI = CHI / (
#                 (types_words_A.get(pair, 0) + types_words_B.get(pair, 0)) *
#                 (types_words_C.get(pair, 0) + types_words_D.get(pair, 0)))
#             if CHI > words_suit_marks.get(word, 0):
#                 words_suit_marks[word] = CHI
#                 words_suit_types[word] = t
#         types = words_suit_types[word]
#         marks = words_suit_marks[word]
#         words_types_marks[types][word] = marks

#     mix_cluster_count = 0
#     for key in words_types_marks.keys():
#         temp = words_types_marks[key]
#         if len(temp) == 0:
#             continue
#         if cluster_map_save[key][2] >= len(data) // 4:
#             mix_cluster_count += 1
#             continue
#         wordcount_record = sorted(temp.items(),
#                                   key=lambda item: item[1],
#                                   reverse=True)
#         # increase the first and second word
#         alpha = 0.005
#         if len(wordcount_record) >= 2:
#             for item in wordcount_record[0:2]:
#                 increase_rate = 1 + alpha * item[1] / cluster_map[i][1][2]
#                 print(item[0], " ", item[1], " increase rate: ", increase_rate)
#                 weight = changeWeight(
#                     weight, all_word, item[0],
#                     1 + alpha * (item[1] / cluster_map[i][1][2]), 0)
#         else:
#             item = wordcount_record[0]
#             increase_rate = 1 + alpha * item[1] / cluster_map[i][1][2]
#             print(item[0], " ", item[1], " increase rate: ", increase_rate)
#             weight = changeWeight(weight, all_word, item[0],
#                                   1 + alpha * (item[1] / cluster_map[i][1][2]),
#                                   0)

#     print("mix cluster count = ", mix_cluster_count)
#     return weight


def changeWeight(weight, all_word, word, change_rate, print_details=0):
    change_array = np.eye(weight.shape[1])
    for i in range(len(all_word)):
        if all_word[i] == word:
            break
    change_array[i][i] = change_rate
    weight = np.dot(weight, change_array)
    return weight

In [10]:
# convenient to train
initial_weight = weight

#### training ####

In [11]:
# each time training from the start point
weight = initial_weight
print_details = 0
for i in range(0, 7):
    print('第%d次' % (i + 1))
    all_labels,CHI_value = clustering(print_details, weight)
    weight = Change(all_labels, weight,CHI_value)

第1次
K_Means算法 n = 17
result of harabaz_score:  157.11294059450546
result of silhouette_score:  0.19870748122081547
Birch算法 n = 17


C:\Users\ym500\Anaconda3\lib\site-packages\sklearn\cluster\birch.py:77: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  node1_dist, node2_dist = dist[[farthest_idx]]


result of harabaz_score:  62.122852005922375
result of silhouette_score:  0.0729000548940345
Agglomerative算法 n = 17
result of harabaz_score:  140.92234777591565
result of silhouette_score:  0.169442554540974
聚类总情况数:  20
最大类的样本数为： 1201
K_Means算法 n = 18
result of harabaz_score:  146.9455253347263
result of silhouette_score:  0.18931955812701534
Birch算法 n = 18
result of harabaz_score:  60.016146800580415
result of silhouette_score:  0.0731660540839528
Agglomerative算法 n = 18
result of harabaz_score:  136.0116953478995
result of silhouette_score:  0.16505641557915351
聚类总情况数:  21
最大类的样本数为： 1147
K_Means算法 n = 19
result of harabaz_score:  147.10371571825536
result of silhouette_score:  0.2002500611233701
Birch算法 n = 19
result of harabaz_score:  57.751826498972434
result of silhouette_score:  0.0735856109542252
Agglomerative算法 n = 19
result of harabaz_score:  131.60825784907516
result of silhouette_score:  0.16709499391150556
聚类总情况数:  19
最大类的样本数为： 1153
K_Means算法 n = 20
result of harabaz_score: 

find_CHI_index_Error: 称京can not find in CHI_value

#### print result ####

In [ ]:
min_value = 0 
for i in range(30,40):
    kmodel = KMeans(n_clusters=i,
                    n_init=10,
                    max_iter=700,
                    init='k-means++',
                   n_jobs=-1).fit(weight)
    harabaz_value = calinski_harabaz_score(weight,kmodel.labels_)
    silhouette_value = silhouette_score(weight,kmodel.labels_)
    print('final value: ',harabaz_value," n:",i)
    if harabaz_value > min_value:
        min_value = harabaz_value
        labels_ = kmodel.labels_
    
details = {}
kind = df[2].tolist()
for i in range(len(data)):
    if labels_[i] not in details:
        details[labels_[i]] = [[data[i]], [kind[i]],[i]]
    else:
        details[labels_[i]][0].append(data[i])
        details[labels_[i]][1].append(kind[i])
        details[labels_[i]][2].append(i)

        
with open("输出.csv","w",encoding="utf-8",newline="") as datacsv:
#dialect为打开csv文件的方式，默认是excel，delimiter="\t"参数指写入的时候的分隔符
    csvwriter = csv.writer(datacsv,dialect = ("excel"))
    #csv文件插入一行数据，把下面列表中的每一项放入一个单元格（可以用循环插入多行）
    for key, list in details.items():
        for i in range(len(list[0])):
            csvwriter.writerow([key, list[1][i], list[0][i]])
        

In [ ]:
print()
for key,value in details.items():
    if key == 1:
        break
word_count = {}
for item in value[2]:
    for string in data_keywords[item].split(" "):
        if string in word_count:
            word_count[string] += 1
        else:
            word_count[string] = 1
word_count_list = sorted(word_count.items(),
                         key=lambda item: item[1],
                         reverse=True)
for i in range(len(word_count_list)):
    print(word_count_list[i])